In [ ]:
%%javascript
/**********************************************************************************************
Known Mathjax Issue with Chrome - a rounding issue adds a border to the right of mathjax markup
https://github.com/mathjax/MathJax/issues/1300
A quick hack to fix this based on stackoverflow discussions: 
http://stackoverflow.com/questions/34277967/chrome-rendering-mathjax-equations-with-a-trailing-vertical-line
**********************************************************************************************/

$('.math>span').css("border-left-color","transparent")

In [ ]:
%reload_ext autoreload
%autoreload 2

# MIDS - w261 Machine Learning At Scale
__Course Lead:__ Dr James G. Shanahan (__email__ Jimi via  James.Shanahan _AT_ gmail.com)

## Assignment - HW9


---
__Name:__  *Jun Luo*   
__Class:__ MIDS w261 (Section *01*, e.g., Fall 2016 Group 1)     
__Email:__  *jun.luo*@iSchool.Berkeley.edu     
__Unit 9:__   

__Due Time:__ HW9 is due on FRIDAY 11/15/2016. 


# Table of Contents <a name="TOC"></a> 

1.  [HW Intructions](#1)   
2.  [HW References](#2)
3.  [HW Problems](#3)   
1.  [HW Introduction](#1)   
2.  [HW References](#2)
3.  [HW  Problems](#3)   
    1.0.  [HW9.0](#1.0)   
    1.0.  [HW9.1](#1.1)   
    1.2.  [HW9.2](#1.2)   
    1.3.  [HW9.3](#1.3)    
    1.4.  [HW9.4](#1.4)    
    1.5.  [HW9.5](#1.5)    
    1.5.  [HW9.6](#1.6)    
   

<a name="1">
# 1 Instructions
[Back to Table of Contents](#TOC)

MIDS UC Berkeley, Machine Learning at Scale
DATSCIW261 ASSIGNMENT #9

Version 2016-11-01 

### INSTRUCTIONS for SUBMISSIONS
Please use the following form for HW submission:

https://docs.google.com/forms/d/1ZOr9RnIe_A06AcZDB6K1mJN4vrLeSmS2PD6Xm3eOiis/viewform?usp=send_form 


### IMPORTANT

HW9 can be completed locally on your computer for most part but will require a cluster of computers for the bigger wikipedia dataset.

### Documents:
* IPython Notebook, published and viewable online.
* PDF export of IPython Notebook.
    
<a name="2">
# 2 Useful References
[Back to Table of Contents](#TOC)

* See async and live lectures for this week
* Data-intensive text processing with MapReduce. San Rafael, CA: Morgan & Claypool Publishers. Chapter 5. 



<a name="3">
# HW Problems
[Back to Table of Contents](#TOC)

<h2 style="color:darkblue">HW 9 Dataset</h2>

Note that all referenced files life in the enclosing directory. [Checkout the Data subdirectory on Dropbox](https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0) or the AWS S3 buckets (details contained each question). 

## 3.  HW9.0  Short answer questions<a name="1.0"></a>
[Back to Table of Contents](#TOC)

__ What is PageRank and what is it used for in the context of web search?__

Per Wikipedia:

PageRank is an algorithm used by Google Search to rank websites in their search engine results. It is a way of measuring the importance of website pages. According to Google:

"PageRank works by counting the number and quality of links to a page to determine a rough estimate of how important the website is. The underlying assumption is that more important websites are likely to receive more links from other websites."

<hr>

__ What modifications have to be made to the webgraph in order to leverage the machinery of Markov Chains to compute the Steady State Distibution? __

To computer the steady state distribution under Perron Frobenius Theorem, the Markov chain process needs to have these two properties: irreducible and aperiodic. 

For web graph:

Irreducible – There is a path from every node to every other node
Aperiodic – The great common divisor of all cycle lengths is 1

Teleportation and dangling node were introduced to satisfy the above well behaved graph properties so that we can leverage the Markov Chain calculation. At any time, the web surfer can randomly jump to a new node. For dangling nodes, their PageRank masses will be re-distributed to the whole graph so that they will not be lost.

<hr>

__ OPTIONAL: In topic-specific pagerank, how can we ensure that the irreducible property is satifsied? (HINT: see HW9.4) __



<hr>


## 3.  HW9.1  MRJob implementation of basic PageRank <a name="1.1"></a>
[Back to Table of Contents](#TOC)

Write a basic MRJob implementation of the iterative PageRank algorithm that takes sparse adjacency lists as input (as explored in HW 7).

Make sure that you implementation utilizes teleportation (1-damping/the number of nodes in the network), and further, distributes the mass of dangling nodes with each iteration so that the output of each iteration is correctly normalized (sums to 1).


[NOTE: The PageRank algorithm assumes that a random surfer (walker), starting from a random web page, chooses the next page to which it will move by clicking at random, with probability d,one of the hyperlinks in the current page. This probability is represented by a so-called *damping factor* d, where d ∈ (0, 1). Otherwise, with probability (1 − d), the surfer jumps to any web page in the network. If a page is a dangling end, meaning it has no outgoing hyperlinks, the random surfer selects an arbitrary web page from a uniform distribution and “teleports” to that page]


As you build your code, use the test data:

> s3://ucb-mids-mls-networks/PageRank-test.txt

Or under the Data Subfolder for HW7 on Dropbox with the same file name. 
> Dropbox: https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0

with teleportation parameter set to 0.15 (1-d, where d, the damping factor is set to 0.85), and crosscheck your work with the true result, displayed in the first image in the [Wikipedia article](https://en.wikipedia.org/wiki/PageRank)
and here for reference are the corresponding PageRank probabilities:
<pre>

A, 0.033
B, 0.384
C, 0.343
D, 0.039
E, 0.081
F, 0.039
G, 0.016
H, 0.016
I, 0.016
J, 0.016
K, 0.016

</pre>

<h2 style="color:darkgreen"> HW 9.1 Implementation </h2>

In [17]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [18]:
%reload_ext autoreload
%autoreload 2

In [19]:
%%writefile PageRankToy.py
## Code goes here

import ast
from mrjob.job import MRJob
from mrjob.protocol import JSONProtocol
from mrjob.step import MRStep

def init_graph(in_file, out_file):
    nodes = []
    with open(in_file, "r") as f:
        for line in f:
            node = line.split('\t')[0]
            if node not in nodes:
                nodes.append(node)
                
            linked_nodes = ast.literal_eval(line.split('\t')[1])
            for k,v in linked_nodes.items():
                if k not in nodes:
                    nodes.append(k)
        
    num_nodes = len(nodes)

    with open(out_file, "w") as fout:
        for n in sorted(nodes):
            links = []
            payload = "{'score': " + str(1.0/num_nodes) + ", "
            
            with open(in_file,"r") as fin:
                for line in fin:
                    node = line.split('\t')[0]
                    linked_nodes = ast.literal_eval(line.split('\t')[1])
                    
                    if n == node:
                        for k,v in linked_nodes.items():
                            links.append(k)
            
            payload += "'links': " + str(links) + "}"
            
            fout.write(n + '\t' + payload +'\n')
            
    return num_nodes

class PageRankToy(MRJob):

    INTERNAL_PROTOCOL = JSONProtocol
    
    def mapper(self, _, line):
        node = line.split('\t')[0]
        payload = ast.literal_eval(line.split('\t')[1])
        
        score = payload['score']        
        links = payload['links']
        yield node, ('links', links) 
        
        p = 0.0
        if len(links) != 0:
            p = float(score)/len(links)
            for n in links:
                yield n,('score', p)
        else:
            #store missing mass in counter
            self.increment_counter('group', 'missing_mass', int(round(score,9)*10e+9))

    def reducer(self, node, payload):
        updated_score = 0
        links = []
        
        for key, value in payload:
            if key == 'links':
                links = value
            elif key == 'score':
                updated_score += value

        new_payload = "{'score': " + str(updated_score) + ", "
        new_payload += "'links': " + str(links) + "}"

        yield node, new_payload
        
        
    def steps(self):
        return ([MRStep(
                    mapper=self.mapper,
                    reducer=self.reducer)
                ]) 
    
if __name__ == '__main__':
    PageRankToy.run() 

Overwriting PageRankToy.py


In [20]:
%%writefile PageRankToy_Driver.py

## Drivers & Runners
#!/usr/bin/python
import sys
import ast
from PageRankToy import PageRankToy, init_graph

input_graph_file = "PageRank-test.txt"
output_graph_file = "graph.txt"

N = init_graph(input_graph_file, output_graph_file)
mr_job = PageRankToy(args=[output_graph_file])

i = 0

#get input parameters
d = float(sys.argv[1]) #damping factor
max_iterations = int(sys.argv[2])
debug = sys.argv[3]

a = 1-d #teleportation factor

while(True):

    p_sum = 0.0
    payload = {}
    
    if debug=="true": print "iteration %d:"%(i+1)
    if debug=="true": print "---------------------------------------------"
    with mr_job.make_runner() as runner: 
        runner.run()
        missing_mass = float(runner.counters()[0]['group']['missing_mass'])/10e+9
        
        
        # stream_output: get access of the output 
        with open(output_graph_file,"w") as f:
            for line in runner.stream_output():
                key,value =  mr_job.parse_output_line(line)
                
                payload = ast.literal_eval(value)
                score = float(payload['score'])
                new_score =  a*(1.0/N) + d*(missing_mass/N + score)
                
                p_sum += new_score
                #print key, value, score, new_score
                
                new_value = "{'score': " + str(new_score) + ", "
                new_value += "'links': " + str(payload['links']) + "}"
                
                if debug=="true": print key, new_value
                f.write(key+'\t'+new_value+'\n')

    missing_mass = float(runner.counters()[0]['group']['missing_mass'])/10e+9
    if debug=="true": print "\nmissing mass: %10.3f"%(missing_mass)
    if debug=="true": print "probability distribution: %10.3f\n\n"%(p_sum)
    
    i = i + 1
    if(i >= max_iterations):
        break

print "======================================================"
print "PageRank after %d iterations with damping factor=%4.2f"%(max_iterations, d)
print "======================================================"
with open(output_graph_file,"r") as f:
    for line in f:
        node = line.split('\t')[0]
        value = ast.literal_eval(line.split('\t')[1])
        for k,v in value.items():
            if k=="score":
                print"%s%8.3f"%(node,v)

Overwriting PageRankToy_Driver.py


In [83]:
## Run Scripts, S3 Sync
## Usage: python driver.py damping_factor iterations debug_flag
!python PageRankToy_Driver.py 0.85 30 false
!echo
!echo "Graph data:"
!cat graph.txt

PageRank after 30 iterations with damping factor=0.85
A   0.033
B   0.384
C   0.344
D   0.039
E   0.081
F   0.039
G   0.016
H   0.016
I   0.016
J   0.016
K   0.016

Graph data:
A	{'score': 0.0327814940127, 'links': []}
B	{'score': 0.383596812817, 'links': ['C']}
C	{'score': 0.343714416648, 'links': ['B']}
D	{'score': 0.0390870930929, 'links': ['A', 'B']}
E	{'score': 0.0808856947411, 'links': ['B', 'D', 'F']}
F	{'score': 0.0390870930929, 'links': ['B', 'E']}
G	{'score': 0.0161694791591, 'links': ['B', 'E']}
H	{'score': 0.0161694791591, 'links': ['B', 'E']}
I	{'score': 0.0161694791591, 'links': ['B', 'E']}
J	{'score': 0.0161694791591, 'links': ['E']}
K	{'score': 0.0161694791591, 'links': ['E']}


<h2 style="color:darkgreen">  HW 9.1 Analysis </h2>




<br><br>

## 3.  HW9.2: Exploring PageRank teleportation and network plots <a name="1.2"></a>
[Back to Table of Contents](#TOC)


* In order to overcome  problems such as disconnected components, the damping factor (a typical value for d is 0.85) can be varied. 
* Using the graph in HW1, plot the test graph (using networkx, https://networkx.github.io/) for several values of the damping parameter alpha, so that each nodes radius is proportional to its PageRank score. 
* In particular you should do this for the following damping factors: [0,0.25,0.5,0.75, 0.85, 1]. 
* Note your plots should look like the following: https://en.wikipedia.org/wiki/PageRank#/media/File:PageRanks-Example.svg

<h2 style="color:darkgreen"> HW 9.2 Implementation </h2>

In [ ]:
## Code goes here
import networkx as nx
import ast
import subprocess

def get_Color(score):
    
    colors = ['#FFFFFF','#E2FFE2','#C6FFC6','#AAFFAA','#8DFF8D','#71FF71','#55FF55','#38FF38','#1CFF1C','#00FF00']
    return colors[int(score*10)]

def draw_Graph(title):
    nodes, sizes, edges, colors,labels = [], [], [], [], {}
    default_node_size = 12000
    
    with open("graph.txt", "r") as f:
        for line in f:
            node = line.split('\t')[0]
            nodes.append(node)
            
            value = ast.literal_eval(line.split('\t')[1])
            for k,v in value.items():
                if k=="score":
                    score = round(float(v),3)
                    labels[node] = node + '\n' + str(score)
                    sizes.append(default_node_size*score)
                    colors.append(get_Color(score))
                elif k=="links":
                    for n in v:
                        edges.append((node,n))

    
    plt.figure(figsize=(8,8))
    plt.title(title)
    plt.axis('off')
    ax = plt.gca()
    ax.autoscale_view('tight')
    
    
    G=nx.DiGraph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    pos=nx.shell_layout(G)

    nx.draw_networkx_nodes(G,pos,
                           nodelist=nodes,
                           node_color=colors,
                           node_size=sizes, alpha=0.5)

    nx.draw_networkx_edges(G,pos,
                           edgelist=edges,
                           width=1,alpha=1,edge_color='grey')

    nx.draw_networkx_labels(G,pos,labels,font_size=10)
    



damping_factors = [0,0.25,0.5,0.75, 0.85, 1]
for d in damping_factors:
    
    cmd = "python PageRankToy_Driver.py " + str(d) + " 30 false"
    subprocess.call(cmd, shell=True)
    title = "Damping Factor = " + str(round(d,2))
    draw_Graph(title)

In [34]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.2 Analysis </h2>




<br><br>

## 3.  HW9.3: Applying PageRank to the Wikipedia hyperlinks network <a name="1.3"></a>
[Back to Table of Contents](#TOC)


* Run your PageRank implementation on the Wikipedia dataset for 5 iterations, and display the top 100 ranked nodes (with alpha = 0.85).
* Run your PageRank implementation on the Wikipedia dataset for 10 iterations, and display the top 100 ranked nodes (with teleportation factor of 0.15).
* Have the top 100 ranked pages changed? Comment on your findings. 
* Plot the pagerank values for the top 100 pages resulting from the 5 iterations run. Then plot the pagerank values for the same 100 pages that resulted from the 10 iterations run.  


<h2 style="color:darkgreen"> HW 9.3 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

 <h2 style="color:darkgreen">  HW 9.3 Analysis </h2>




<br><br>

## 3.  HW9.4: Topic-specific PageRank implementation using MRJob <a name="1.4"></a>
[Back to Table of Contents](#TOC)

Modify your PageRank implementation to produce a topic specific PageRank implementation, as described in:

http://www-cs-students.stanford.edu/~taherh/papers/topic-sensitive-pagerank.pdf

Note in this article that there is a special caveat to ensure that the transition matrix is irreducible.   
This caveat lies in footnote 3 on page 3:
```
	A minor caveat: to ensure that M is irreducible when p
	contains any 0 entries, nodes not reachable from nonzero
	nodes in p should be removed. In practice this is not problematic.
```
and must be adhered to for convergence to be guaranteed.   

Run topic specific PageRank on the following randomly generated network of 100 nodes:

> s3://ucb-mids-mls-networks/randNet.txt (also available on Dropbox)

which are organized into ten topics, as described in the file:

> s3://ucb-mids-mls-networks/randNet_topics.txt  (also available on Dropbox)

Since there are 10 topics, your result should be 11 PageRank vectors (one for the vanilla PageRank implementation in 9.1, and one for each topic with the topic specific implementation). Print out the top ten ranking nodes and their topics for each of the 11 versions, and comment on your result. Assume a teleportation factor of 0.15 in all your analyses.

One final and important comment here:  please consider the requirements for irreducibility with topic-specific PageRank. In particular, the literature ensures irreducibility by requiring that nodes not reachable from in-topic nodes be removed from the network.

This is not a small task, especially as it it must be performed separately for each of the (10) topics.

So, instead of using this method for irreducibility, please comment on why the literature's method is difficult to implement, and what what extra computation it will require.   

Then for your code, please use the alternative, non-uniform damping vector:

```
vji = beta*(1/|Tj|); if node i lies in topic Tj

vji = (1-beta)*(1/(N - |Tj|)); if node i lies outside of topic Tj
```
for beta in (0,1) close to 1. 

With this approach, you will not have to delete any nodes. If beta > 0.5, PageRank is topic-sensitive, and if beta < 0.5, the PageRank is anti-topic-sensitive. For any value of beta irreducibility should hold, so please try beta=0.99, and perhaps some other values locally, on the smaller networks.

<h2 style="color:darkgreen"> HW 9.4 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.4 Analysis </h2>




<br><br>

<center><div class='jumbotron'><h3 style='color:darkblue'>---------  OPTIONAL QUESTIONS SECTION --------</h3></div></center>

## 3.  HW9.5: (OPTIONAL) Applying topic-specific PageRank to Wikipedia <a name="1.5"></a>
[Back to Table of Contents](#TOC)


Here you will apply your topic-specific PageRank implementation to Wikipedia, defining topics (very arbitrarily) for each page by the length (number of characters) of the name of the article mod 10, so that there are 10 topics. 

* Once again, print out the top ten ranking nodes and their topics for each of the 11 versions, and comment on your result. Assume a teleportation factor of 0.15 in all your analyses. Run for 10 iterations.
* Plot the pagerank values for the top 100 pages resulting from the 5 iterations run in HW 9.3. 
* Then plot the pagerank values for the same 100 pages that result from the topic specific pagerank after 10 iterations run. 
* Comment on your findings. 

<h2 style="color:darkgreen"> HW 9.5 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.5 Analysis </h2>




<br><br>

## 3.  HW9.6:  (OPTIONAL) TextRank <a name="1.6"></a>
[Back to Table of Contents](#TOC)


* What is TextRank? Describe the main steps in the algorithm. Why does TextRank work?
* Implement TextRank in MrJob for keyword phrases (not just unigrams) extraction using co-occurrence based similarity measure with with sizes of N = 2 and 3. And evaluate your code using the following example using precision, recall, and FBeta (Beta=1):
```
"Compatibility of systems of linear constraints over the set of natural numbers
Criteria of compatibility of a system of linear Diophantine equations, strict 
inequations, and nonstrict inequations are considered. Upper bounds for
components of a minimal set of solutions and algorithms of construction of 
minimal generating sets of solutions for all types of systems are given. 
These criteria and the corresponding algorithms for constructing a minimal 
supporting set of solutions can be used in solving all the considered types of 
systems and systems of mixed types." 
```
* The extracted keywords should in the following set:
```
linear constraints, linear diophantine equations, natural numbers, non-strict inequations, strict inequations, upper bounds
```

<h2 style="color:darkgreen"> HW 9.6 Implementation </h2>

In [1]:
## Code goes here

In [2]:
## Drivers & Runners

In [3]:
## Run Scripts, S3 Sync

<h2 style="color:darkgreen">  HW 9.6 Analysis </h2>




<br><br>

<center><div class='jumbotron'><h2 style='color:green'>-------  END OF HWK 9 --------</h2></div></center>